In [1]:
import os
os.chdir('D:\\Gnan\\DA\\KMU\\Prediction-COVID-19')
from utils import *
from model import *
from trainer import *

from tqdm.notebook import tqdm
from torch.optim.adam import Adam
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import warnings
warnings.filterwarnings('ignore')

In [2]:
path = 'Data/Variants/SIR/alpha_delta/*.csv'
dic_files = Load_files.load_files(path, -18, -16)
data = dic_files['대구']
data

,stdDay,Susceptible,Alpha_Infected,Alpha_Recovered,Alpha_Dead,Alpha_alpha,Alpha_beta,Alpha_gamma,Delta_Infected,Delta_Recovered,Delta_Dead,Delta_alpha,Delta_beta,Delta_gamma
0,2021-04-12,2380065,39.65,163.36,2.00,0.059242,0.058260,0.005296,0.32,0.00,0.00,0.564647,0.000000,0.0
1,2021-04-13,2380056,39.47,165.67,2.21,0.072737,0.079807,0.000000,0.50,0.00,0.00,0.441681,0.000000,0.0
2,2021-04-14,2380045,39.18,168.82,2.21,0.119906,0.037519,0.000000,0.72,0.00,0.00,0.501913,0.000000,0.0
3,2021-04-15,2380027,42.39,170.29,2.21,0.024628,0.084218,0.000000,1.08,0.00,0.00,0.074358,0.000000,0.0
4,2021-04-16,2380023,39.86,173.86,2.21,0.078574,0.052684,0.000000,1.16,0.00,0.00,0.207690,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,2021-08-11,2376752,12.67,486.64,3.05,0.030942,0.132597,0.000000,814.69,726.09,0.84,0.045715,0.052928,0.0
122,2021-08-12,2376713,11.38,488.32,3.05,0.050350,0.163445,0.000000,808.62,769.21,0.84,0.067316,0.059039,0.0
123,2021-08-13,2376656,10.09,490.18,3.05,0.081696,0.136769,0.000000,815.03,816.95,0.84,0.096082,0.043459,0.0
124,2021-08-14,2376574,9.53,491.56,3.05,0.060128,0.072403,0.000000,857.51,852.37,0.84,0.063482,0.020653,0.0


In [3]:
learning_rates_list = [1e-3, 1e-4]
patiences_list = [20, 50]
num_layers_list = [1, 2, 4]
batch_sizes_list = [32, 64]
hidden_sizes_list = [8, 16, 32]
dropout_list = [0.25]
criterions_list = [nn.MSELoss(), criterion2, criterion3]

hyperparameter_dict = {}
i = 0

for lr in learning_rates_list:
    for patience in patiences_list:
        for num_layers in num_layers_list:
            for batch_sizes in batch_sizes_list:
                for hidden_size in hidden_sizes_list:    
                    for dropout in dropout_list:
                        for criterion in criterions_list:
                            hyperparameter_dict[i] = [lr, patience, num_layers, batch_sizes, hidden_size, dropout, criterion]
                            i += 1
                        
print(i)

216


In [4]:
input_size = 3
sequence_length = 60
num_epochs = 10000

df = Prepare_df.processing(data, 'stdDay', 'Delta_alpha')

x = df.iloc[:, 0:]
y = df.iloc[:,:1]

ms = MinMaxScaler()
ss = StandardScaler()

ss.fit(x)
ms.fit(y)

MinMaxScaler()

In [5]:
best_model = {}
models_list = ['seq2seq_RNN', 'seq2seq_LSTM', 'seq2seq_GRU', 
               'seq2seq_BiRNN', 'seq2seq_BiLSTM', 'seq2seq_BiGRU']

for num_model in range(6):
    rmse_min = np.inf
    for hyperparameter in tqdm(hyperparameter_dict.values(), 
                               desc = f'대구_{models_list[num_model]}', position = 0):
        
        lr = hyperparameter[0]
        patience = hyperparameter[1]
        num_layers = hyperparameter[2]
        batch_size = hyperparameter[3]
        hidden_size = hyperparameter[4]
        dropout = hyperparameter[5]
        criterion = hyperparameter[6]
        #print('-------------------------------------------------------------------------------------')
        #print('lr:', lr, ' patience:', patience, ' num_layers:', num_layers, 
        #      ' batch_size:', batch_size, ' hidden_size:', hidden_size, ' dropout:', dropout)
    
        x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 37, 60, 14, batch_size, 'mtm')
        
        if num_model == 0:
            model = RNN_encoder_decoder(input_size = input_size, 
                                        hidden_size = hidden_size,
                                        num_layers = num_layers, 
                                        dropout = dropout,
                                        device = device).to(device)
            
        elif num_model == 1:
            model = LSTM_encoder_decoder(input_size = input_size, 
                                         hidden_size = hidden_size,
                                         num_layers = num_layers, 
                                         dropout = dropout,
                                         device = device).to(device)
            
        elif num_model == 2:
            model = GRU_encoder_decoder(input_size = input_size, 
                                        hidden_size = hidden_size,
                                        num_layers = num_layers, 
                                        dropout = dropout,
                                        device = device).to(device)
            
        elif num_model == 3:
            model = BiRNN_encoder_decoder(input_size = input_size, 
                                            hidden_size = hidden_size,
                                            num_layers = num_layers, 
                                            dropout = dropout,
                                            device = device).to(device)

        elif num_model == 4:
            model = BiLSTM_encoder_decoder(input_size = input_size, 
                                              hidden_size = hidden_size,
                                              num_layers = num_layers, 
                                              dropout = dropout,
                                              device = device).to(device)

        elif num_model == 5:
            model = BiGRU_encoder_decoder(input_size = input_size, 
                                            hidden_size = hidden_size,
                                            num_layers = num_layers, 
                                            dropout = dropout,
                                            device = device).to(device)

        optimizer = Adam(model.parameters(), lr = lr)
    
        loss_list, model, epoch = Trainer.Many_to_Many(train_loader, 
                                                       test_loader, 
                                                       model, 
                                                       criterion, 
                                                       optimizer, 
                                                       num_epochs, 
                                                       patience,
                                                       14,
                                                       device)
    
        label_y, predicted, first_label_y, first_predicted, all_predicted = predict_mtm(model, df, x_ss, y_ms, 
                                                                                        14, 0.5, device)
    
        if rmse_min > rmse(label_y[-14:], predicted[-14:]):
            rmse_min = rmse(label_y[-14:], predicted[-14:])
            best_model[models_list[num_model]] = [rmse_min, model, loss_list, epoch, lr, patience, 
                                                  num_layers, batch_size, hidden_size, dropout, criterion]

대구_seq2seq_RNN:   0%|          | 0/216 [00:00<?, ?it/s]

대구_seq2seq_LSTM:   0%|          | 0/216 [00:00<?, ?it/s]

대구_seq2seq_GRU:   0%|          | 0/216 [00:00<?, ?it/s]

대구_seq2seq_BiRNN:   0%|          | 0/216 [00:00<?, ?it/s]

대구_seq2seq_BiLSTM:   0%|          | 0/216 [00:00<?, ?it/s]

대구_seq2seq_BiGRU:   0%|          | 0/216 [00:00<?, ?it/s]

In [6]:
best_model

{'seq2seq_RNN': [0.0531263573907896,
  RNN_encoder_decoder(
    (RNN_encoder): RNN_encoder(
      (rnn): RNN(3, 32, num_layers=4, batch_first=True, dropout=0.25)
    )
    (RNN_decoder): RNN_decoder(
      (rnn): RNN(3, 32, num_layers=4, batch_first=True, dropout=0.25)
      (linear): Linear(in_features=32, out_features=1, bias=True)
    )
  ),
  [0.03203935921192169,
   0.02211720682680607,
   0.020103303715586662,
   0.022870806977152824,
   0.021649474278092384,
   0.01912129670381546,
   0.019978895783424377,
   0.01876419596374035,
   0.0181150883436203,
   0.019086118787527084,
   0.019077718257904053,
   0.018641149625182152,
   0.018507663160562515,
   0.01726570911705494,
   0.017373045906424522,
   0.01763463206589222,
   0.018199460580945015,
   0.01783737540245056,
   0.01800251565873623,
   0.017471598461270332,
   0.017027597874403,
   0.017711514607071877,
   0.016975751146674156,
   0.01677073910832405,
   0.01751057803630829,
   0.016704091802239418,
   0.0165706910192

In [7]:
for key, values in best_model.items():
    save_model(values[1].state_dict(), f"model/SIR/delta/{key}.pth")
    save_hyperparameter(values[3:], f"hyperparameter/SIR/delta/{key}.pkl")